# 第2回：転移学習と不均衡データへの挑戦

## 学習目標

- 皮膚病変画像（DermaMNIST）を用い、実務上の課題である「データの偏り」への対策を学ぶ。
- 学習済みモデルを用いた「転移学習」により、高精度なモデルを構築する。

---
## 転移学習とは

転移学習は、大規模なデータセット（ImageNetなど）で学習済みのモデルを、別のタスクに再利用する手法です。ゼロから学習するよりも少ないデータで高精度を達成できます。

自転車に乗れる人がバイクを学ぶとき、バランス感覚やハンドル操作の知識を活かせます。同様に、ImageNetで100万枚の画像から学んだ「エッジや形状を認識する能力」を、数千枚の医療画像の分類に転用できます。

## クラス不均衡とは

医療データでは、正常サンプルが多く異常サンプルが少ない「クラス不均衡」がよく発生します。そのまま学習すると、少数クラスの認識精度が低下します。

100人のクラスで95人が健康、5人が病気だとします。「全員健康」と予測するだけで95%の精度になりますが、肝心の病気を見逃してしまいます。これがクラス不均衡の問題です。

## クラス重み（class_weight）による対策

少数クラスのサンプルに大きな重みを与えることで、モデルが少数クラスも適切に学習できるようにします。

試験勉強で苦手科目に多くの時間を割くのと同じ発想です。「病気」のサンプルを間違えたときのペナルティを大きくすることで、モデルは少数派のパターンもしっかり学習します。

### クラス重みの計算式

scikit-learnの `compute_class_weight('balanced', ...)` は、以下の式でクラス重みを計算します：

$$\text{重み}_i = \frac{\text{全サンプル数}}{\text{クラス数} \times \text{クラス } i \text{ のサンプル数}}$$

**例**: 全1000サンプル、2クラス（健康: 950、病気: 50）の場合
- 健康の重み = 1000 / (2 × 950) = 0.53
- 病気の重み = 1000 / (2 × 50) = 10.0

病気クラスの重みが約19倍になり、少数派の誤分類がより大きなペナルティを受けるようになります。

In [ ]:
# 環境セットアップ
!pip install medmnist -q
import sys, os
!rm -rf /tmp/MedMNIST-Exercise
!git clone https://github.com/kshimoji8/MedMNIST-Exercise-Public.git /tmp/MedMNIST-Exercise -q
sys.path.insert(0, '/tmp/MedMNIST-Exercise')
sys.modules.pop('exercise_logic', None)
import exercise_logic
exercise_logic.initialize_environment()
print("✓ セットアップが完了しました。")

# 皮膚画像データのロード（DermaMNIST）
(x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess('dermamnist')

# クラス不均衡の可視化と重み計算
import numpy as np
from sklearn.utils import class_weight

y_train_flat = y_train.flatten()
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_flat), y=y_train_flat)
class_weights_dict = dict(enumerate(weights))
print("算出されたクラス重み:", class_weights_dict)

In [ ]:
# 転移学習モデル（MobileNetV2）を構築
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=len(info['label']), 
    model_type='transfer'
)

# クラス重みを適用して学習
history = model.fit(
    x_train, y_train, 
    epochs=10, 
    validation_split=0.1, 
    class_weight=class_weights_dict, 
    batch_size=32
)

In [ ]:
# 評価
exercise_logic.plot_history(history)
exercise_logic.show_evaluation_reports(model, x_test, y_test, info['label'])

---
## 練習問題

### 練習1: 転移学習を使わない場合と比較してみよう

モデル構築の `model_type='transfer'` を `model_type='simple'` に変更して、精度を比較してください。

```python
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=len(info['label']), 
    model_type='simple'  # ← 'transfer' から変更
)
```

**観察ポイント:**
- 転移学習ありとなしで、精度にどの程度の差がありますか？
- 学習曲線の収束速度に違いはありますか？

### 練習2: クラス重みを使わない場合と比較してみよう

`model.fit()` から `class_weight=class_weights_dict` を削除して、精度を比較してください。

```python
history = model.fit(
    x_train, y_train, 
    epochs=10, 
    validation_split=0.1, 
    # class_weight=class_weights_dict,  # ← この行をコメントアウト
    batch_size=32
)
```

**観察ポイント:**
- 全体の精度（accuracy）はどう変わりますか？
- 混同行列を見て、少数クラスの認識精度はどう変わりますか？

---
## 考察課題

以下の点について考えてみましょう：

1. **転移学習の有効性**: なぜ一般的な画像（ImageNet）で学習したモデルが、医療画像の分類にも有効なのでしょうか？

2. **クラス不均衡の影響**: 皮膚がんの検出AIで、悪性腫瘍（少数）を見逃すことと、良性を悪性と誤診することでは、どちらがより深刻な問題でしょうか？

3. **精度の解釈**: 7クラス分類で「精度85%」と言われた場合、その数値だけで性能を判断できますか？クラスごとの精度を確認する重要性について考えてください。

---
## まとめ

本講義で学んだ内容：

- **転移学習**: 大規模データで学習済みのモデルを別タスクに再利用する手法
- **MobileNetV2**: 軽量で高性能な学習済みモデル
- **クラス不均衡**: 医療データでよく見られる、クラス間のサンプル数の偏り
- **クラス重み**: 少数クラスに大きな重みを与えて、偏りを補正する手法
- **実務的な課題**: 精度だけでなく、各クラスの認識性能を確認する重要性

次回は「Grad-CAM」を用いて、AIが画像のどこに注目しているかを可視化します。